https://medium.com/usf-msds/a-langchain-chatbot-using-pdfs-6b83dfa904de

In [1]:
import re
import json
from google.cloud import vision
from google.cloud import storage

In [2]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [3]:
GCS_BUCKET_NAME="rio-rag-platform-document-storage"

Application Default Credentials (ADC) è una strategia utilizzata dalle librerie di autenticazione di Google per trovare automaticamente le credenziali in base all'ambiente delle applicazioni. Le librerie di autenticazione rendono queste credenziali disponibili alle librerie client di Cloud e alle librerie client delle API di Google. 

In [4]:
def upload_file_to_gcs(bucket_name, file_object):
  """
  Funzione per caricare un file su Google Cloud Storage.

  Args:
    bucket_name (str): Nome del bucket GCS.
    file_object (file): Oggetto file da caricare.

  Returns:
    str: URL del file caricato.
  """

  # Creare un client GCS
  storage_client = storage.Client(
    project=os.getenv('GCP_PROJECT_ID')
  )

  bucket = storage_client.bucket(bucket_name)

  # Caricare il file
  blob = bucket.blob(file_object.name)
  blob.upload_from_file(file_object)

  # Restituire l'URL del file
  return blob.public_url

In [5]:

# file_object = st.file_uploader("Seleziona un file") #da streamlit 
file_object = open(r"C:\Users\ELAFACRB1\OneDrive - NTT DATA EMEAL\Desktop\Catalogo dei Finalisti IV Award 2023 - InnovationVillage.pdf", "rb") #da python
file_object

<_io.BufferedReader name='C:\\Users\\ELAFACRB1\\OneDrive - NTT DATA EMEAL\\Desktop\\Catalogo dei Finalisti IV Award 2023 - InnovationVillage.pdf'>

In [6]:
object_url = upload_file_to_gcs(GCS_BUCKET_NAME, file_object)
object_url

'https://storage.googleapis.com/rio-rag-platform-document-storage/C%3A%5CUsers%5CELAFACRB1%5COneDrive%20-%20NTT%20DATA%20EMEAL%5CDesktop%5CCatalogo%20dei%20Finalisti%20IV%20Award%202023%20-%20InnovationVillage.pdf'

In [7]:
def async_detect_document(gcs_source_uri, gcs_destination_uri):
    """Asynchronous OCR with PDF/TIFF as source files on GCS.

    Args:
        gcs_source_uri (str): The GCS URI of the source document.
        gcs_destination_uri (str): The GCS URI for storing the OCR results.

    Returns:
        list: A list of extracted text from the document.
    """

    # Define MIME type for PDF documents
    mime_type = "application/pdf"

    # Set batch size for asynchronous processing
    batch_size = 1

    # Create a Vision API client
    client = vision.ImageAnnotatorClient()

    # Define the feature type for document text detection
    feature = vision.Feature(type_=vision.Feature.Type.DOCUMENT_TEXT_DETECTION)

    # Define GCS source and input configuration
    gcs_source = vision.GcsSource(uri=gcs_source_uri)
    input_config = vision.InputConfig(gcs_source=gcs_source, mime_type=mime_type)

    # Define GCS destination and output configuration
    gcs_destination = vision.GcsDestination(uri=gcs_destination_uri)
    output_config = vision.OutputConfig(
        gcs_destination=gcs_destination, batch_size=batch_size
    )

    # Create an asynchronous annotation request
    async_request = vision.AsyncAnnotateFileRequest(
        features=[feature], input_config=input_config, output_config=output_config
    )

    # Submit the asynchronous request and get the operation
    operation = client.async_batch_annotate_files(requests=[async_request])

    # Wait for the operation to finish (timeout: 7 minutes)
    print("Waiting for the operation to finish.")
    operation.result(timeout=420)

    # Initialize Storage client
    storage_client = storage.Client()

    # Extract bucket name and prefix from GCS destination URI
    match = re.match(r"gs://([^/]+)/(.+)", gcs_destination_uri)
    bucket_name = match.group(1)
    prefix = match.group(2)

    # Get the bucket
    bucket = storage_client.get_bucket(bucket_name)

    # List all blobs (files) in the specified prefix
    blob_list = [
        blob
        for blob in list(bucket.list_blobs(prefix=prefix))
        if not blob.name.endswith("/")
    ]

    # Extract text from each OCR response and store in a list
    docs = []
    for filename in blob_list:
        json_string = filename.download_as_bytes().decode("utf-8")
        response = json.loads(json_string)
        response = response["responses"][0]
        annotation = response["fullTextAnnotation"]
        docs.append(annotation['text'])

    return docs

In [8]:
filenames = [r'gs://rio-rag-platform-document-storage/C:\Users\ELAFACRB1\OneDrive - NTT DATA EMEAL\Desktop\Catalogo dei Finalisti IV Award 2023 - InnovationVillage.pdf']

output_path = 'gs://rio-rag-platform-document-output/'


In [9]:
docs = []

for filename in filenames:
  docs.extend(async_detect_document(filename,output_path))

PermissionDenied: 403 Cloud Vision API has not been used in project progetti-poc before or it is disabled. Enable it by visiting https://console.developers.google.com/apis/api/vision.googleapis.com/overview?project=progetti-poc then retry. If you enabled this API recently, wait a few minutes for the action to propagate to our systems and retry. [links {
  description: "Google developers console API activation"
  url: "https://console.developers.google.com/apis/api/vision.googleapis.com/overview?project=progetti-poc"
}
, reason: "SERVICE_DISABLED"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "vision.googleapis.com"
}
metadata {
  key: "consumer"
  value: "projects/progetti-poc"
}
]